In [2]:
#from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report,accuracy_score 
import numpy as np
import pandas as pd

In [360]:
df=pd.read_csv('training_data.csv')
df = df.fillna('N') #populating each player's last round empty cells with letter N
print(df)

      playerID  roundID state next_state      Fac1      Fac2      Fac3  \
0           96        1    CC         CC  0.852809  0.000000  0.001421   
1           96        2    CC         CC  1.764813  0.000646  0.000000   
2           96        3    CC         CC  1.779278  0.000000  0.000231   
3           96        4    CC         CC  1.783243  0.000000  0.006609   
4           96        5    CC         CC  1.430331  0.000000  0.020980   
5           96        6    CC         CC  1.235734  0.000000  0.005232   
6           96        7    CC         CC  2.057244  0.000000  0.026242   
7           96        8    CC         CC  1.972266  0.000000  0.050355   
8           96        9    CC         CC  1.524039  0.000000  0.064471   
9           97       11    CC         CC  0.309282  0.409891  0.312269   
10          97       12    CC         CC  0.308497  0.169956  0.575895   
11          97       13    CC         CC  0.030269  1.239617  0.384072   
12          97       14    CC         

In [361]:
nextState=[]
dic={"C":0, "D":1,"CC":0,"CD":1,"DC":2,"DD":3} #mapping letters to numbers as random forest needs features to be numbers 

j=1 #for making rounds range from 1 to 10 for each player
last_j=0 #for assigning roundId to the last row of the df as the if condition will fail for the last round and assign 1 to j

own_state=[]
opp_state=[]

joint_coop_rate_so_far=[]
joint_coop_counter=0

self_coop_rate_so_far=[]
self_coop_counter=0

self_betrayal_rate_so_far=[]
self_betrayal_counter=0

opp_coop_rate_so_far=[]
opp_coop_counter=0

opp_betrayal_rate_so_far=[]
opp_betrayal_counter=0

for i in range(0,len(df)):
    own_state.append(dic[df["state"][i][0]])
    opp_state.append(dic[df["state"][i][1]])
    df["next_state"][i] = dic[df["next_state"][i][0]]
    df["roundID"][i] = j
    
    if(df["state"][i]=="CC"):
        joint_coop_counter+=1
        
    if(df["state"][i][0]=="C"):
        self_coop_counter+=1
        
    if(df["state"][i][0]=="D"):
        self_betrayal_counter+=1
    
    if(df["state"][i][1]=="C"):
        opp_coop_counter+=1
    
    if(df["state"][i][1]=="D"):
        opp_betrayal_counter+=1
    
    df["state"][i] = dic[df["state"][i]]
    
    joint_coop_rate_so_far.append(joint_coop_counter/10)
    self_coop_rate_so_far.append(self_coop_counter/10)
    self_betrayal_rate_so_far.append(self_betrayal_counter/10)
    opp_coop_rate_so_far.append(opp_coop_counter/10)
    opp_betrayal_rate_so_far.append(opp_betrayal_counter/10)
    
    if(i<len(df)-1 and df["playerID"][i]==df["playerID"][i+1]):
        j+=1
        last_j=j
    else:
        j=1
        joint_coop_counter=0
        self_coop_counter=0
        self_betrayal_counter=0
        opp_coop_counter=0
        opp_betrayal_counter=0

df["roundID"][len(df)-1] = last_j

c:\users\seahorse\myenv\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\seahorse\myenv\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\seahorse\myenv\lib\site-packages\ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\seahorse\myenv\lib\site-packages\ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

In [362]:
df=df.assign(own_state=own_state)
df=df.assign(opp_state=opp_state)
df=df.assign(joint_coop_rate_so_far=joint_coop_rate_so_far)
df=df.assign(self_coop_rate_so_far=self_coop_rate_so_far)
df=df.assign(self_betrayal_rate_so_far=self_betrayal_rate_so_far)
df=df.assign(opp_coop_rate_so_far=opp_coop_rate_so_far)
df=df.assign(opp_betrayal_rate_so_far=opp_betrayal_rate_so_far)
df.to_csv("transformed.csv", encoding='utf-8', index=False)

In [391]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics, cross_validation
from sklearn import datasets

df_features = df
df_features = df_features.drop(["playerID"],axis=1)
#df_features = df_features.drop(["roundID"],axis=1)
#df_features = df_features.drop(["state"],axis=1)
df_features = df_features.drop(["next_state"],axis=1)
df_features = df_features.drop(["Fac1"],axis=1)
df_features = df_features.drop(["Fac2"],axis=1)
df_features = df_features.drop(["Fac3"],axis=1)
#df_features = df_features.drop(["Fac4"],axis=1)
#df_features = df_features.drop(["Fac5"],axis=1)
#df_features = df_features.drop(["Fac6"],axis=1)
df_features = df_features.drop(["opp_state"],axis=1)
#df_features = df_features.drop(["self_coop_rate_so_far"],axis=1)
#df_features = df_features.drop(["self_betrayal_rate_so_far"],axis=1)

df_target = df["next_state"]
df_target= df_target.astype('int')

predicted = cross_validation.cross_val_predict(LogisticRegression(), df_features, df_target, cv=10)

print(metrics.roc_auc_score(df_target, predicted))
print(metrics.classification_report(df_target, predicted)) 

0.7421072796934867
             precision    recall  f1-score   support

          0       0.76      0.79      0.78       551
          1       0.73      0.69      0.71       450

avg / total       0.75      0.75      0.75      1001



In [383]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics, cross_validation
from sklearn import datasets
import xgboost as xgb

df_features = df
df_features = df_features.drop(["playerID"],axis=1)
#df_features = df_features.drop(["roundID"],axis=1)
#df_features = df_features.drop(["state"],axis=1)
df_features = df_features.drop(["next_state"],axis=1)
df_features = df_features.drop(["Fac1"],axis=1)
df_features = df_features.drop(["Fac2"],axis=1)
df_features = df_features.drop(["Fac3"],axis=1)
#df_features = df_features.drop(["Fac4"],axis=1)
#df_features = df_features.drop(["Fac5"],axis=1)
#df_features = df_features.drop(["Fac6"],axis=1)
df_features = df_features.drop(["opp_state"],axis=1)
#df_features = df_features.drop(["self_coop_rate_so_far"],axis=1)
#df_features = df_features.drop(["self_betrayal_rate_so_far"],axis=1)

df_target = df["next_state"]
df_target= df_target.astype('int')

predicted = cross_validation.cross_val_predict(GradientBoostingClassifier(), df_features, df_target, cv=10)

print(metrics.roc_auc_score(df_target, predicted))
print(metrics.classification_report(df_target, predicted)) 

0.7040875176446865
             precision    recall  f1-score   support

          0       0.74      0.73      0.73       551
          1       0.67      0.68      0.68       450

avg / total       0.71      0.71      0.71      1001



In [384]:
from sklearn import metrics, cross_validation
from sklearn import datasets
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


df_features = df
df_features = df_features.drop(["playerID"],axis=1)
#df_features = df_features.drop(["roundID"],axis=1)
#df_features = df_features.drop(["state"],axis=1)
df_features = df_features.drop(["next_state"],axis=1)
df_features = df_features.drop(["Fac1"],axis=1)
df_features = df_features.drop(["Fac2"],axis=1)
df_features = df_features.drop(["Fac3"],axis=1)
#df_features = df_features.drop(["Fac4"],axis=1)
#df_features = df_features.drop(["Fac5"],axis=1)
#df_features = df_features.drop(["Fac6"],axis=1)
df_features = df_features.drop(["opp_state"],axis=1)
#df_features = df_features.drop(["self_coop_rate_so_far"],axis=1)
#df_features = df_features.drop(["self_betrayal_rate_so_far"],axis=1)

df_target = df["next_state"]
df_target= df_target.astype('int')

predicted = cross_validation.cross_val_predict(GradientBoostingClassifier(), df_features, df_target, cv=10)

print(metrics.roc_auc_score(df_target, predicted))
print(metrics.classification_report(df_target, predicted)) 

0.7040875176446865
             precision    recall  f1-score   support

          0       0.74      0.73      0.73       551
          1       0.67      0.68      0.68       450

avg / total       0.71      0.71      0.71      1001



In [390]:
from sklearn import metrics, cross_validation
from xgboost import XGBClassifier

df_features = df
df_features = df_features.drop(["playerID"],axis=1)
#df_features = df_features.drop(["roundID"],axis=1)
#df_features = df_features.drop(["state"],axis=1)
df_features = df_features.drop(["next_state"],axis=1)
df_features = df_features.drop(["Fac1"],axis=1)
df_features = df_features.drop(["Fac2"],axis=1)
df_features = df_features.drop(["Fac3"],axis=1)
#df_features = df_features.drop(["Fac4"],axis=1)
#df_features = df_features.drop(["Fac5"],axis=1)
#df_features = df_features.drop(["Fac6"],axis=1)
df_features = df_features.drop(["opp_state"],axis=1)
#df_features = df_features.drop(["self_coop_rate_so_far"],axis=1)
#df_features = df_features.drop(["self_betrayal_rate_so_far"],axis=1)

df_target = df["next_state"]
df_target= df_target.astype('int')

predicted = cross_validation.cross_val_predict(XGBClassifier(n_estimators=500), df_features.values, df_target.values, cv=10)

print(metrics.roc_auc_score(df_target, predicted))
print(metrics.classification_report(df_target, predicted)) 

c:\users\seahorse\myenv\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
c:\users\seahorse\myenv\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
c:\users\seahorse\myenv\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
c:\users\seahorse\myenv\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in fu

0.6909578544061303
             precision    recall  f1-score   support

          0       0.72      0.72      0.72       551
          1       0.66      0.66      0.66       450

avg / total       0.69      0.69      0.69      1001



c:\users\seahorse\myenv\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [340]:
from sklearn.svm import SVC
from sklearn import metrics, cross_validation
from sklearn import datasets

df_features = df
df_features = df_features.drop(["playerID"],axis=1)
#df_features = df_features.drop(["roundID"],axis=1)
#df_features = df_features.drop(["state"],axis=1)
df_features = df_features.drop(["next_state"],axis=1)
#df_features = df_features.drop(["coop_rate_so_far"],axis=1)
df_features = df_features.drop(["Fac1"],axis=1)
df_features = df_features.drop(["Fac2"],axis=1)
df_features = df_features.drop(["Fac3"],axis=1)
#df_features = df_features.drop(["Fac4"],axis=1)
#df_features = df_features.drop(["Fac5"],axis=1)
#df_features = df_features.drop(["Fac6"],axis=1)
#df_features = df_features.drop(["opp_state"],axis=1)
#df_features = df_features.drop(["self_coop_rate_so_far"],axis=1)
#df_features = df_features.drop(["self_betrayal_rate_so_far"],axis=1)

df_target = df["next_state"]
#df_target= df_target.astype('int')

predicted = cross_validation.cross_val_predict(SVC(gamma='auto'), df_features, df_target, cv=10)

print(metrics.accuracy_score(df_target, predicted))
print(metrics.classification_report(df_target, predicted)) 

0.7112887112887113
             precision    recall  f1-score   support

          C       0.76      0.69      0.73       551
          D       0.66      0.74      0.70       450

avg / total       0.72      0.71      0.71      1001



Performance Expectations

In [12]:
#timeseries cross validation training and testing
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, oob_score=True, random_state=123456,max_features='auto')

df_target = df["next_state"]
df_target= df_target.astype('int')

df_features = df
df_features = df_features.drop(["next_state"],axis=1)
df_features = df_features.drop(["playerID"],axis=1)
#df_features = df_features.drop(["Fac2"],axis=1)
#df_features = df_features.drop(["roundID"],axis=1)


# timeseries construction
folds = 998
tscv = TimeSeriesSplit(n_splits=folds)

# validation and performance evaluation
foldid = 0
totacc = 0.
ytlog = []
yplog = []
totalcount=0

for train_index, test_index in tscv.split(df_features):
    foldid += 1
    X_train, X_test = df_features.iloc[train_index],df_features.iloc[test_index]
    y_train, y_test = df_target[train_index], df_target[test_index]
    

    rf.fit(X_train.values, y_train)
    if(foldid>500):
        y_pred = rf.predict(X_test.values)
        acc = accuracy_score(y_pred, y_test)
        totacc += acc
        totalcount+=1
        ytlog += list(y_test)
        yplog += list(y_pred)
        print(str(foldid)+" "+str(acc))
    print(foldid)

print(totacc)
print("Average Accuracy: %0.3f" % (totacc / totalcount))
print(classification_report(ytlog, yplog))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [11]:
#timeseries cross validation training and testing
from sklearn.model_selection import TimeSeriesSplit
from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()

df_target = df["next_state"]
df_target= df_target.astype('int')

df_features = df
df_features = df_features.drop(["next_state"],axis=1)
df_features = df_features.drop(["playerID"],axis=1)
#df_features = df_features.drop(["Fac2"],axis=1)
#df_features = df_features.drop(["roundID"],axis=1)


# timeseries construction
folds = 998
tscv = TimeSeriesSplit(n_splits=folds)

# validation and performance evaluation
foldid = 0
totacc = 0.
ytlog = []
yplog = []
totalcount=0

for train_index, test_index in tscv.split(df_features):
    foldid += 1
    X_train, X_test = df_features.iloc[train_index],df_features.iloc[test_index]
    y_train, y_test = df_target[train_index], df_target[test_index]
    

    clf.fit(X_train.values, y_train)
    
    if(foldid>500):
        y_pred = clf.predict(X_test.values)
        acc = accuracy_score(y_pred, y_test)
        totacc += acc
        totalcount+=1
        ytlog += list(y_test)
        yplog += list(y_pred)
        print(str(foldid)+" "+str(acc))
    print(foldid)

print(totacc)
print("Average Accuracy: %0.3f" % (totacc / totalcount))
print(classification_report(ytlog, yplog))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [27]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import TimeSeriesSplit

df_target = df["next_state"]
df_target= df_target.astype('int')

df_features = df
df_features = df_features.drop(["next_state"],axis=1)
df_features = df_features.drop(["playerID"],axis=1)

# timeseries construction
folds = 3000
tscv = TimeSeriesSplit(n_splits=folds)


# validation and performance evaluation
foldid = 0
totacc = 0.
ytlog = []
yplog = []
newacc=0
for train_index, test_index in tscv.split(df_features):
    foldid += 1
    X_train, X_test = df_features.iloc[train_index],df_features.iloc[test_index]
    y_train, y_test = df_target[train_index], df_target[test_index]
    
    
    clf.fit(X_train.values, y_train)

    
    y_pred = clf.predict(X_test.values)
    acc = accuracy_score(y_pred, y_test)
    
    if(foldid>2500):
        totacc += acc
        ytlog += list(y_test)
        yplog += list(y_pred)    
    print(str(foldid)+" "+str(acc))
    
print(totacc)
print("Average Accuracy: %0.3f" % (totacc / 500))
print(classification_report(ytlog, yplog))

1 1.0
2 1.0
3 1.0
4 1.0
5 1.0
6 1.0
7 0.0
8 1.0
9 1.0
10 1.0
11 1.0
12 1.0
13 1.0
14 1.0
15 1.0
16 1.0
17 0.0
18 1.0
19 1.0
20 1.0
21 1.0
22 0.0
23 1.0
24 0.0
25 0.0
26 1.0
27 0.0
28 0.0
29 0.0
30 1.0
31 0.0
32 1.0
33 0.0
34 1.0
35 0.0
36 0.0
37 0.0
38 1.0
39 0.0
40 1.0
41 0.0
42 1.0
43 1.0
44 0.0
45 1.0
46 0.0
47 0.0
48 0.0
49 1.0
50 0.0
51 0.0
52 1.0
53 0.0
54 0.0
55 1.0
56 0.0
57 0.0
58 0.0
59 0.0
60 0.0
61 0.0
62 1.0
63 1.0
64 0.0
65 1.0
66 1.0
67 1.0
68 0.0
69 1.0
70 0.0
71 0.0
72 1.0
73 1.0
74 1.0
75 1.0
76 0.0
77 1.0
78 1.0
79 1.0
80 0.0
81 0.0
82 0.0
83 0.0
84 0.0
85 0.0
86 0.0
87 0.0
88 0.0
89 0.0
90 0.0
91 0.0
92 1.0
93 0.0
94 0.0
95 0.0
96 0.0
97 0.0
98 0.0
99 0.0
100 0.0
101 0.0
102 0.0
103 0.0
104 0.0
105 1.0
106 0.0
107 0.0
108 0.0
109 0.0
110 0.0
111 0.0
112 1.0
113 0.0
114 0.0
115 0.0
116 1.0
117 1.0
118 1.0
119 1.0
120 1.0
121 0.0
122 0.0
123 0.0
124 1.0
125 1.0
126 1.0
127 0.0
128 1.0
129 1.0
130 1.0
131 0.0
132 1.0
133 1.0
134 1.0
135 1.0
136 1.0
137 1.0
138 1.0
139 

1035 1.0
1036 1.0
1037 1.0
1038 1.0
1039 0.0
1040 1.0
1041 0.0
1042 0.0
1043 0.0
1044 0.0
1045 1.0
1046 0.0
1047 0.0
1048 0.0
1049 0.0
1050 1.0
1051 1.0
1052 1.0
1053 0.0
1054 0.0
1055 0.0
1056 1.0
1057 0.0
1058 1.0
1059 0.0
1060 1.0
1061 0.0
1062 0.0
1063 0.0
1064 0.0
1065 0.0
1066 0.0
1067 0.0
1068 0.0
1069 0.0
1070 1.0
1071 0.0
1072 1.0
1073 1.0
1074 0.0
1075 1.0
1076 0.0
1077 0.0
1078 1.0
1079 0.0
1080 1.0
1081 1.0
1082 1.0
1083 1.0
1084 1.0
1085 1.0
1086 1.0
1087 1.0
1088 1.0
1089 0.0
1090 0.0
1091 1.0
1092 1.0
1093 0.0
1094 0.0
1095 1.0
1096 0.0
1097 1.0
1098 0.0
1099 0.0
1100 1.0
1101 1.0
1102 1.0
1103 1.0
1104 1.0
1105 1.0
1106 1.0
1107 1.0
1108 1.0
1109 0.0
1110 1.0
1111 0.0
1112 0.0
1113 0.0
1114 1.0
1115 1.0
1116 0.0
1117 0.0
1118 0.0
1119 0.0
1120 1.0
1121 1.0
1122 1.0
1123 1.0
1124 1.0
1125 1.0
1126 1.0
1127 1.0
1128 1.0
1129 0.0
1130 1.0
1131 1.0
1132 1.0
1133 1.0
1134 1.0
1135 1.0
1136 1.0
1137 1.0
1138 1.0
1139 0.0
1140 1.0
1141 1.0
1142 1.0
1143 1.0
1144 1.0
1145 1.0
1

1951 1.0
1952 0.0
1953 0.0
1954 0.0
1955 1.0
1956 1.0
1957 1.0
1958 1.0
1959 1.0
1960 0.0
1961 1.0
1962 0.0
1963 1.0
1964 1.0
1965 1.0
1966 1.0
1967 1.0
1968 1.0
1969 1.0
1970 0.0
1971 1.0
1972 1.0
1973 1.0
1974 1.0
1975 1.0
1976 1.0
1977 1.0
1978 1.0
1979 1.0
1980 0.0
1981 1.0
1982 1.0
1983 1.0
1984 1.0
1985 1.0
1986 1.0
1987 1.0
1988 1.0
1989 1.0
1990 0.0
1991 0.0
1992 0.0
1993 1.0
1994 0.0
1995 1.0
1996 1.0
1997 0.0
1998 0.0
1999 0.0
2000 0.0
2001 0.0
2002 1.0
2003 0.0
2004 0.0
2005 0.0
2006 0.0
2007 0.0
2008 0.0
2009 1.0
2010 1.0
2011 0.0
2012 0.0
2013 1.0
2014 0.0
2015 0.0
2016 0.0
2017 0.0
2018 0.0
2019 1.0
2020 0.0
2021 1.0
2022 0.0
2023 0.0
2024 0.0
2025 0.0
2026 0.0
2027 0.0
2028 0.0
2029 0.0
2030 0.0
2031 0.0
2032 0.0
2033 0.0
2034 0.0
2035 0.0
2036 0.0
2037 0.0
2038 0.0
2039 0.0
2040 0.0
2041 0.0
2042 0.0
2043 0.0
2044 0.0
2045 1.0
2046 1.0
2047 0.0
2048 0.0
2049 0.0
2050 0.0
2051 0.0
2052 0.0
2053 0.0
2054 0.0
2055 1.0
2056 1.0
2057 0.0
2058 0.0
2059 0.0
2060 0.0
2061 0.0
2

2863 1.0
2864 1.0
2865 1.0
2866 0.0
2867 0.0
2868 0.0
2869 1.0
2870 1.0
2871 0.0
2872 0.0
2873 1.0
2874 0.0
2875 1.0
2876 0.0
2877 0.0
2878 0.0
2879 1.0
2880 1.0
2881 0.0
2882 0.0
2883 0.0
2884 0.0
2885 0.0
2886 0.0
2887 0.0
2888 1.0
2889 1.0
2890 0.0
2891 0.0
2892 1.0
2893 0.0
2894 0.0
2895 0.0
2896 0.0
2897 1.0
2898 0.0
2899 1.0
2900 1.0
2901 0.0
2902 0.0
2903 1.0
2904 1.0
2905 0.0
2906 0.0
2907 0.0
2908 1.0
2909 0.0
2910 0.0
2911 1.0
2912 0.0
2913 0.0
2914 1.0
2915 1.0
2916 0.0
2917 1.0
2918 1.0
2919 1.0
2920 1.0
2921 1.0
2922 0.0
2923 0.0
2924 1.0
2925 0.0
2926 0.0
2927 1.0
2928 1.0
2929 1.0
2930 1.0
2931 1.0
2932 1.0
2933 1.0
2934 1.0
2935 1.0
2936 0.0
2937 1.0
2938 1.0
2939 1.0
2940 1.0
2941 1.0
2942 1.0
2943 1.0
2944 1.0
2945 0.0
2946 0.0
2947 0.0
2948 1.0
2949 1.0
2950 1.0
2951 1.0
2952 1.0
2953 0.0
2954 1.0
2955 1.0
2956 1.0
2957 1.0
2958 1.0
2959 1.0
2960 0.0
2961 0.0
2962 1.0
2963 1.0
2964 1.0
2965 0.0
2966 0.0
2967 0.0
2968 0.0
2969 0.0
2970 0.0
2971 0.0
2972 1.0
2973 1.0
2

c:\users\seahorse\myenv\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [55]:
print("Average Accuracy: %0.3f" % (totacc))

Average Accuracy: 0.000
